In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from secret_scrimmage_lineup_guesser.data_processing import clean_data

df = pd.read_csv('../data/raw_data.csv')

# Defines who can play which positions, and who gets preferences for which position
position_preferences = {
    1: ["dai_dai_ames", "jalen_warley", "andrew_rohde"],
    2: ["isaac_mckneely", "andrew_rohde", "ishan_sharma", "taine_murray"],
    3: ["taine_murray", "ishan_sharma", "andrew_rohde", "jalen_warley", "tj_power", "elijah_saunders"],
    4: ["elijah_saunders", "tj_power", "jacob_cofie"],
    5: ["blake_buchanan", "jacob_cofie", "elijah_saunders"],
}
df = clean_data(df)
df


,name,min,sec,plus_minus,name_lower,time,round_time
0,Blake Buchanan,26,17,2,blake_buchanan,26.811628,27
1,Jalen Warley,16,7,0,jalen_warley,16.440612,16
2,Elijah Saunders,34,31,-13,elijah_saunders,35.210452,35
3,Isaac McKneely,29,48,-11,isaac_mckneely,30.398980,30
4,TJ Power,21,7,-18,tj_power,21.541112,22
5,Jacob Cofie,15,45,3,jacob_cofie,16.066575,16
6,Andrew Rohde,23,53,-11,andrew_rohde,24.363388,24
7,Dai Dai Ames,21,37,-11,dai_dai_ames,22.051162,22
8,Taine Murray,6,59,1,taine_murray,7.123698,7
9,Ishan Sharma,0,57,3,ishan_sharma,0.969095,1


In [3]:
import pandas as pd
from itertools import product
from typing import List, Dict
from collections import defaultdict

def generate_all_valid_lineups(
    position_preferences: Dict[int, List[str]]
) -> List[Dict[int, str]]:
    """
    Generate all possible lineups based on position preferences.

    Args:
        position_preferences (dict): A dictionary where keys are positions (1-5) and values are
                                     lists of players ordered by preference for that position.

    Returns:
        List[Dict[int, str]]: A list of dictionaries, each representing a valid lineup where keys
                              are positions and values are player names.
    """
    def backtrack(position: int, current_lineup: Dict[int, str], used_players: set, all_lineups: List[Dict[int, str]]):
        if position > 5:
            # All positions have been assigned
            all_lineups.append(current_lineup.copy())
            return
        
        for player in position_preferences.get(position, []):
            if player not in used_players:
                current_lineup[position] = player
                used_players.add(player)
                
                backtrack(position + 1, current_lineup, used_players, all_lineups)
                
                # Backtrack
                used_players.remove(player)
                del current_lineup[position]
    
    all_lineups = []
    backtrack(1, {}, set(), all_lineups)
    return all_lineups

def get_invalid_pairs(lineups: List[Dict[int, str]], position_preferences: Dict[int, List[str]]) -> List[Dict[int, str]]:
    """
    Filter lineups to ensure that no player is assigned in a way that violates the
    preference hierarchy across different positions.

    Args:
        lineups (List[Dict[int, str]]): List of generated lineups to be filtered.
        position_preferences (dict): Position preferences as defined earlier.

    Returns:
        List[Dict[int, str]]: Filtered list of valid lineups respecting preference hierarchy.
    """
    # Create a player to preferences mapping for quick lookup
    player_preferences = defaultdict(dict)
    for pos, players in position_preferences.items():
        for rank, player in enumerate(players):
            player_preferences[player][pos] = rank
    
    invalid_pairs = []
    for lineup in lineups:
        for pos1, player1 in lineup.items():
            for pos2, player2 in lineup.items():
                if pos1 <= pos2:
                    continue
                # Mark invalid if player1 preferred over player2 for pos2 and vice versa
                pref11 = player_preferences[player1].get(pos1, float('inf'))
                pref12 = player_preferences[player1].get(pos2, float('inf'))
                pref21 = player_preferences[player2].get(pos1, float('inf'))
                pref22 = player_preferences[player2].get(pos2, float('inf'))
                if pref11 > pref21 and pref12 < pref22:
                    invalid_pairs.append(((player1, pos1), (player2, pos2)))
    return list(set(invalid_pairs))

position_preferences = {
    1: ["dai_dai_ames", "jalen_warley", "andrew_rohde"],
    2: ["isaac_mckneely", "andrew_rohde", "ishan_sharma", "taine_murray"],
    3: ["taine_murray", "ishan_sharma", "andrew_rohde", "jalen_warley", "tj_power", "elijah_saunders"],
    4: ["elijah_saunders", "tj_power", "jacob_cofie"],
    5: ["blake_buchanan", "jacob_cofie", "elijah_saunders"],
}

all_lineups = generate_all_valid_lineups(position_preferences)
invalid_pairs = get_invalid_pairs(all_lineups, position_preferences)




In [4]:
invalid_pairs

[(('andrew_rohde', 3), ('taine_murray', 2)),
 (('ishan_sharma', 3), ('taine_murray', 2)),
 (('andrew_rohde', 3), ('ishan_sharma', 2)),
 (('tj_power', 4), ('elijah_saunders', 3)),
 (('elijah_saunders', 5), ('jacob_cofie', 4)),
 (('jalen_warley', 3), ('andrew_rohde', 1))]

In [5]:
from secret_scrimmage_lineup_guesser.optimization import run_opt

lineup_df = run_opt(df, position_preferences, invalid_pairs, overall_pm=-11, time_limit=100)
lineup_df


,min,1,2,3,4,5,plus_minus
0,1,jalen_warley,isaac_mckneely,elijah_saunders,jacob_cofie,blake_buchanan,2.0
1,2,jalen_warley,isaac_mckneely,andrew_rohde,elijah_saunders,blake_buchanan,3.0
2,3,jalen_warley,isaac_mckneely,tj_power,elijah_saunders,jacob_cofie,-3.0
3,4,dai_dai_ames,isaac_mckneely,tj_power,elijah_saunders,jacob_cofie,3.0
4,5,dai_dai_ames,isaac_mckneely,andrew_rohde,tj_power,elijah_saunders,-3.0
5,6,jalen_warley,ishan_sharma,tj_power,elijah_saunders,blake_buchanan,3.0
6,7,dai_dai_ames,isaac_mckneely,jalen_warley,elijah_saunders,blake_buchanan,3.0
7,8,dai_dai_ames,isaac_mckneely,jalen_warley,tj_power,blake_buchanan,-3.0
8,9,andrew_rohde,isaac_mckneely,taine_murray,elijah_saunders,jacob_cofie,-3.0
9,10,dai_dai_ames,andrew_rohde,elijah_saunders,jacob_cofie,blake_buchanan,3.0


In [6]:
from secret_scrimmage_lineup_guesser.optimization import get_positional_min_max

results = get_positional_min_max(df, position_preferences, invalid_pairs, overall_pm=-11, time_limit=10)
display(results)


100%|██████████| 10/10 [02:55<00:00, 17.58s/it]


,player,1,2,3,4,5
0,blake_buchanan,0,0,0,0,27
1,jalen_warley,0-16,0,0-16,0,0
2,elijah_saunders,0,0,3-16,6-31,3-13
3,isaac_mckneely,0,30,0,0,0
4,tj_power,0,0,4-17,5-16,0
5,jacob_cofie,0,0,0,5-16,0-11
6,andrew_rohde,2-18,2-10,0-20,0,0
7,dai_dai_ames,22,0,0,0,0
8,taine_murray,0,0-4,0-7,0,0
9,ishan_sharma,0,0-1,0-1,0,0


In [7]:
tmp1 = run_opt(
    df,
    position_preferences,
    invalid_pairs,
    overall_pm=-11,
    time_limit=10,
    objective=("jacob_cofie", 5, True),
)
tmp2 = run_opt(
    df,
    position_preferences,
    invalid_pairs,
    overall_pm=-11,
    time_limit=10,
    objective=("jacob_cofie", 5, False),
)

In [8]:
for i, tmp in enumerate([tmp1, tmp2]):
    print(f"Cofie only/mostly played at the {i+4}:")

    cofie = (tmp[4] == "jacob_cofie") | (tmp[5] == "jacob_cofie")
    buchanan = tmp[5] == "blake_buchanan"
    saunders = (
        (tmp[4] == "elijah_saunders")
        | (tmp[5] == "elijah_saunders")
        | (tmp[3] == "elijah_saunders")
    )

    # Create all possible combinations of the three boolean arrays
    combinations = [
        (~cofie & ~buchanan & ~saunders, "None"),
        (cofie & ~buchanan & ~saunders, "Only Cofie"),
        (~cofie & buchanan & ~saunders, "Only Buchanan"),
        (~cofie & ~buchanan & saunders, "Only Saunders"),
        (cofie & buchanan & ~saunders, "Cofie & Buchanan"),
        (cofie & ~buchanan & saunders, "Cofie & Saunders"),
        (~cofie & buchanan & saunders, "Buchanan & Saunders"),
        (cofie & buchanan & saunders, "All three"),
    ]

    for mask, label in combinations:
        filtered = tmp[mask]
        if len(filtered) > 0:
            print(f"\n{label}:")
            print(f"Minutes: {len(filtered)}")
            print(f"Plus/minus: {filtered['plus_minus'].sum()}")
    print("\n\n")

Cofie only/mostly played at the 4:

Only Saunders:
Minutes: 13
Plus/minus: -13.0

Cofie & Buchanan:
Minutes: 5
Plus/minus: 2.0

Buchanan & Saunders:
Minutes: 11
Plus/minus: -1.0

All three:
Minutes: 11
Plus/minus: 1.0



Cofie only/mostly played at the 5:

Only Cofie:
Minutes: 3
Plus/minus: 1.0

Only Saunders:
Minutes: 2
Plus/minus: -6.0

Cofie & Buchanan:
Minutes: 2
Plus/minus: 1.0

Cofie & Saunders:
Minutes: 8
Plus/minus: -8.0

Buchanan & Saunders:
Minutes: 22
Plus/minus: -8.0

All three:
Minutes: 3
Plus/minus: 9.0





In [9]:
results.to_csv("../data/results.csv", index=False)